## Backtesting 

- Trades at close
- Commission = 0.125%
- SL = TP = 5%
- Cash = 1 M
- N Shares = 50
- No taxes
- No leverages
- Margin acc = 50%
- Borrow rate = 0.25%
- Time frame = 5 m

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta
from dataclasses import dataclass
import optuna

sns.set_theme()

/Users/sofia/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
data = pd.read_csv('aapl_5m_train.csv').dropna()
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0


In [19]:
rsi_indicator = ta.momentum.RSIIndicator(data.Close, window=20)
data['rsi'] = rsi_indicator.rsi()

In [20]:
historic = data.dropna()
historic['buy_signal'] = historic["rsi"] < 20
historic['sell_signal'] = historic["rsi"] > 80

/var/folders/39/pg899vtj5774718bjd1zhn6m0000gn/T/ipykernel_29397/1825628898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['buy_signal'] = historic["rsi"] < 20
/var/folders/39/pg899vtj5774718bjd1zhn6m0000gn/T/ipykernel_29397/1825628898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['sell_signal'] = historic["rsi"] > 80


In [21]:
def get_portfolio_value(cash: float, long_ops: list[Operation], short_ops: list[Operation], 
                        current_price: float, n_shares: int) -> float:
   
    val = cash 

    # Add long position values
    val += len(long_ops) * current_price * n_shares 

    # Add short position values

    return val

In [22]:
COM = 0.125 / 100
SL = 0.1
TP = 0.1
BORROW_RATE = 0.25 / 100

cash = 1_000_000
n_shares = 200

active_long_positions: list[Operation] = []

portfolio_value = [cash]

for i, row in historic.iterrows():
    # This only works for long positions
    
    # Close positions
    for position in active_long_positions.copy():
        if row.Close > position.take_profit or row.Close < position.stop_loss:
            cash += row.Close * position.n_shares * (1 - COM)
            active_long_positions.remove(position)
    # --- BUY ---
    # Check signal
    if not row.buy_signal:
        portfolio_value.append(get_portfolio_value(cash, active_long_positions, [], row.Close, n_shares))
        continue
    # Do we have enough cash?
    if cash < row.Close * n_shares * (1 + COM):
        portfolio_value.append(get_portfolio_value(cash, active_long_positions, [], row.Close, n_shares))
        continue
    # Discount the cost
    cash -= row.Close * n_shares * (1 + COM)
    # Save the operation as active position
    active_long_positions.append(
        Operation(
        time=row.Datetime,
        price=row.Close,
        take_profit=row.Close * (1 + TP),
        stop_loss=row.Close * (1 - SL),
        n_shares=n_shares,
        type="LONG"
        )
    )

    portfolio_value.append(get_portfolio_value(cash, active_long_positions, [], row.Close, n_shares))

cash += row.Close * len(active_long_positions) * n_shares * (1 - COM)
active_long_positions = []

In [23]:
cash, len(active_long_positions)

(1175409.6117202505, 0)